In [343]:
#imports
import pandas as pd
import numpy as np
import panel as pn

from statistics import mean
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show

from fractions import Fraction

#data gathering
notes = pd.read_excel('../data/all_sections_notes.xlsx')
student_data = pd.read_excel('../data/student_information_filtered.xlsx')


In [344]:
#Student Data filtering

student_data = student_data[student_data['NUMERO'].notna()]
student_data.drop(columns={'EPS', 'Comentarios','Falta por consentimiento'}, axis=1, inplace=True)
student_data['DOCUMENTO '] = pd.to_numeric(student_data['DOCUMENTO '])
student_data.columns

Index(['NUMERO', 'CSBRIDGE SEC', 'ESTUDIANTE', 'Asistió', 'DOCUMENTO ',
       'GENERO', 'COLEGIO', 'GRADO', 'EMAIL ESTUDIANTE', 'CORREO ACUDIENTE',
       'ESPAÑOL', 'MATEMATICAS', 'PROMEDIO NOTAS', 'ESCALA', 'PUNTAJE',
       'RESULTADO', 'EMAIL CONFIRMACION '],
      dtype='object')

In [345]:
#We rename the columns to make them easier to work with
notes.rename(columns={
    #WEEK 1
    'OrgDefinedId':'user',
    'Secciones':'sec',
    'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1AM',
    'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1PM',
    'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2AM',
    'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2PM',
    'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3AM',
    'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3PM',
    'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4AM',
    'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4PM',

    'Semana 1\Week1 Subtotal Numerator':'week_1_numerator',
    'Semana 1\Week1 Subtotal Denominator':'week_1_denominator',
    
    #WEEK 2
    'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5PM',
    'Entrega de la mañana (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5AM',
    'Entrega de la mañana (D6) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'6AM',

    'Semana 2\Week 2 Subtotal Numerator':'week_2_numerator',
    'Semana 2\Week 2 Subtotal Denominator':'week_2_denominator',

    'Calculated Final Grade Numerator':'final_numerator',
    'Calculated Final Grade Denominator':'final_denominator'
}, inplace = True)

#we change the type to a numeric float64 from 'object'
#Week 1
notes['1AM'] = pd.to_numeric(notes['1AM'])
notes['1PM'] = pd.to_numeric(notes['1PM'])
notes['2AM'] = pd.to_numeric(notes['2AM'])
notes['2PM'] = pd.to_numeric(notes['2PM'])
notes['3AM'] = pd.to_numeric(notes['3AM'])
notes['3PM'] = pd.to_numeric(notes['3PM'])
notes['4AM'] = pd.to_numeric(notes['4AM'])
notes['4PM'] = pd.to_numeric(notes['4PM'])
#Week 2
notes['5AM'] = pd.to_numeric(notes['5AM'])
notes['5PM'] = pd.to_numeric(notes['5PM'])
notes['6AM'] = pd.to_numeric(notes['6AM'])



notes['sec'].replace(['Sección No. 1', 'Sección No. 2', 'Sección No. 3','Sección No. 4','Sección No. 5','Sección No. 6','Sección No. 7',
'Sección No. 8','Sección No. 9','Sección No. 10 - Las morcillitas'], [1,2,3,4,5,6,7,8,9,10], inplace=True)

notes.fillna(0.0, inplace=True)
notes['user'] = pd.to_numeric(notes['user'])

notes.drop(columns={'Adjusted Final Grade Numerator', 'Adjusted Final Grade Denominator', 'Username'}, axis=1, inplace=True)


In [346]:
data = pd.merge(notes, student_data, how='inner', left_on='user', right_on='DOCUMENTO ')

data.drop(columns={'EMAIL ESTUDIANTE',
       'CORREO ACUDIENTE', 'ESPAÑOL', 'MATEMATICAS', 'PROMEDIO NOTAS',
       'ESCALA', 'PUNTAJE', 'RESULTADO', 'EMAIL CONFIRMACION ','CSBRIDGE SEC','Asistió','Email','DOCUMENTO '}, axis=1, inplace=True)


#we recalculate the numerator and denominator for every student, as the data didn't take NaN into account.
for index in data.index:
        #week 1
        a = (sum([data.loc[index, '1AM'],
        data.loc[index, '1PM'],
        data.loc[index, '2AM'],
        data.loc[index, '2PM'],
        data.loc[index, '3AM'],
        data.loc[index, '3PM'],
        data.loc[index, '4AM'],
        data.loc[index, '4PM']]))
        b = 8.0

        data.loc[index, 'week_1_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'week_1_denominator'] = Fraction(a/b).denominator

        #week 2
        a = (sum([data.loc[index, '5AM'],
                data.loc[index, '5PM'],
                data.loc[index, '6AM']]))
        b = 3.0

        data.loc[index, 'week_2_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'week_2_denominator'] = Fraction(a/b).denominator

        #final values, this operation might come as redundant at first, but it is quite important for final averages. Either way, i might 
        #optimize it in the future.
        a = (sum([data.loc[index, '1AM'],
        data.loc[index, '1PM'],
        data.loc[index, '2AM'],
        data.loc[index, '2PM'],
        data.loc[index, '3AM'],
        data.loc[index, '3PM'],
        data.loc[index, '4AM'],
        data.loc[index, '4PM'],
        data.loc[index, '5AM'],
        data.loc[index, '5PM'],
        data.loc[index, '6AM']]))
        b = 11.0
        
        data.loc[index, 'week_2_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'week_2_denominator'] = Fraction(a/b).denominator

data


,user,Last Name,First Name,sec,1AM,1PM,2AM,2PM,3AM,3PM,...,Entrega de la Final del curso Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>,week_2_numerator,week_2_denominator,final_numerator,final_denominator,NUMERO,ESTUDIANTE,GENERO,COLEGIO,GRADO
0,1000594810,GONZALEZ VARGAS,JOHAN SEBASTIAN,3.0,0.0,0.0,3.5,5.0,2.5,0.0,...,0.0,7.574236e+15,4.503600e+15,18.5,25.0,62,JOHAN SEBASTIAN GONZALEZ VARGAS,H,COLEGIO HELADIA MEJIA I.E.D.,11.0
1,101051396,martinez montiel,luisa fernanda,7.0,0.0,5.0,3.5,5.0,0.0,5.0,...,0.0,7.000000e+00,2.000000e+00,38.5,40.0,118,LUISA FERNANDA MARTINEZ MONTIEL,M,SAN JOSE I E D,11.0
2,1010840777,López Rugeles,María José,1.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.0,1.791204e+15,5.629500e+14,35.0,35.0,24,MARÍA JOSÉ LÓPEZ RUGELES,M,COLEGIO SANTA FRANCISCA ROMANA,10.0
3,1010961769,Cortés Mosquera,María Andrea,4.0,5.0,5.0,5.0,5.0,3.5,5.0,...,5.0,4.964195e+15,1.125900e+15,53.5,55.0,52,MARÍA ANDREA CORTÉS MOSQUERA,M,GIMNASIO CAMPESTRE LOS SAUCES,10.0
4,1010962618,Pico,Samuel Pico,5.0,0.0,5.0,5.0,5.0,5.0,5.0,...,0.0,5.117727e+15,1.125900e+15,50.0,50.0,28,SAMUEL PICO,H,GIMNASIO CAMPESTRE BETH SHALOM,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1141314542,mayorga sanchez,cristal mariana,8.0,0.0,3.5,4.8,5.0,5.0,3.3,...,5.0,7.778945e+15,2.251800e+15,43.0,50.0,119,CRISTAL MARIANA MAYORGA SANCHEZ,M,SAN JOSE I E D,11.0
95,1141715139,Bermúdez,Juan David,2.0,5.0,5.0,5.0,5.0,5.0,3.5,...,0.0,5.168904e+15,1.125900e+15,50.5,55.0,93,JUAN DAVID BERMÚDEZ,H,COLEGIO CRISTIANO GRACIA Y AMOR,9.0
96,1145924881,CASTELLANOS FONTECHA,MIGUEL ANGEL,1.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.117727e+15,1.125900e+15,55.0,55.0,58,MIGUEL ANGEL CASTELLANOS FONTECHA,H,LICEO EL ENCUENTRO,9.0
97,665101,KIM,ISAAC,2.0,5.0,3.5,5.0,5.0,5.0,3.5,...,0.0,5.322436e+15,1.125900e+15,52.0,55.0,59,ISAAC KIM,H,LICEO EL ENCUENTRO,10.0


Average grade per day

In [347]:
concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM']
def average_per_day(df):
    concept_averages = []
    for concept in concept_checks:
        concept_averages.append(round(data[concept].sum()/(len(data)), 2))

    avg_concept_notes_source = ColumnDataSource(data=dict(concept_checks = concept_checks, concept_averages = concept_averages, color = Category20c[len(concept_checks)]))
    concept_notes_figure = figure(x_range = concept_checks, y_range=(0,5), height=400, title="Average notes per concept check", tools="")
    concept_notes_figure.vbar(x='concept_checks', top='concept_averages', width=0.45, color='color', legend_field="concept_averages", source=avg_concept_notes_source)

    concept_notes_figure.xgrid.grid_line_color = None
    concept_notes_figure.legend.orientation = "horizontal"
    concept_notes_figure.legend.location = "top_left"
    return concept_notes_figure

show(average_per_day(data))

Average grade per gender

In [348]:
concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM']
gender = ['H','M']
def grade_per_gender(data):
    x = [(concept, g) for concept in concept_checks for g in gender]
    
    concept_averages = []
    for concept in concept_checks:
        concept_averages.append(data.query("GENERO == 'H'")[concept].sum()/len(data.query("GENERO == 'H'")))
        concept_averages.append(data.query("GENERO == 'M'")[concept].sum()/len(data.query("GENERO == 'M'")))
    source = ColumnDataSource(data=dict(x=x, counts=concept_averages))

    concept_gender_figure = figure(x_range=FactorRange(*x),y_range=(0,5), height=400, title="Average grade per gender per concept check",
           toolbar_location=None, tools="")
    concept_gender_figure.vbar(x='x', top='counts', width=0.9, source=source,
    color=factor_cmap('x', palette=['#005067','#FEB9C6'], factors=gender, start=1, end=2))

    return concept_gender_figure

show(grade_per_gender(data))

Percentage of submitted works

In [349]:
concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM']
def submitted_percentage(data):
    concept_percent = []
    for concept in concept_checks:
        concept_percent.append(round((len(data[data[concept]!=0])/len(data))*100, 2))
    
    source = ColumnDataSource(data=dict(concept_checks = concept_checks, concept_percent = concept_percent, color = Category20c[len(concept_percent)]))
    concept_notes_figure = figure(x_range = concept_checks, y_range=(0,100), height=400, title="Submission percentage per concept check", tools="")
    concept_notes_figure.vbar(x='concept_checks', top='concept_percent', width=0.45, color='color', legend_field="concept_percent", source=source)

    concept_notes_figure.xgrid.grid_line_color = None
    concept_notes_figure.legend.orientation = "horizontal"
    concept_notes_figure.legend.location = "top_left"
    return concept_notes_figure

show(submitted_percentage(data))

Web Server Display Template

In [350]:
#Layout using Template
#call to previous functions
submitted_percentage = submitted_percentage(data)

template = pn.template.FastListTemplate(
    title='CS Bridge - Grade Data.',
    main=[pn.Column(pn.Row(pn.pane.Bokeh(submitted_percentage),pn.pane.Bokeh(average_per_day(data)))), 
        pn.Column(pn.Row(grade_per_gender(data)))],
    accent_base_color="#2f3e46",
    header_background="#354f52",
)
template.show()

Launching server at http://localhost:49660
